# Motivating Example: Why Penalized Constrained Regression?

This notebook demonstrates the value of combining **penalization** with **economic constraints** in cost estimation.

## The Problem

Traditional regression methods like OLS can produce coefficients that are:
- **Economically impossible** (e.g., learning curves > 100%) 
- **Unstable** when data is limited or noisy

Caution / Note: ***If there is "negative" learning e.g >100% then you are missing an indepedent variable like capacity constraints or economic order quantity violations, catostrophic events. Always check for those independent variables before applying constraints***
What is the checklist?
## Methods Compared

| Method | Penalization | Constraints | Description |
|--------|-------------|-------------|-------------|
| **OLS** | None | None | Baseline - unconstrained least squares |
| **RidgeCV** | L2 (Ridge) | None | Penalization only - helps with overfitting |
| **PCReg (Constraints Only)** | None | Yes | Constraints only - ensures economic validity |
| **PCReg-GCV** | L1/L2 (Elastic Net) | Yes | Both - the best of both worlds |

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import TransformedTargetRegressor

import penalized_constrained as pcreg

# For presentation plots
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-whitegrid')

## Load Data

We use simulated manufacturing lot data with:
- **lot_midpoint**: Cumulative unit midpoint of the lot
- **lot_quantity**: Number of units in the lot
- **observed_cost**: Per-unit cost (with realistic noise)

In [2]:
# Load the data
DIR = Path('output_v2')
df = pd.read_csv(DIR / 'motivational_example_data.csv')

# Split into train and test
df_train = df.query("lot_type == 'train'").copy()
df_test = df.query("lot_type == 'test'").copy()

X_train = df_train[['lot_midpoint', 'lot_quantity']]
y_train = df_train['observed_cost']

X_test = df_test[['lot_midpoint', 'lot_quantity']]
y_test = df_test['observed_cost']

print(f"Training samples: {len(df_train)}")
print(f"Test samples: {len(df_test)}")
print(f"\nTrue parameters:")
print(f"  T1 = {df['T1_true'].iloc[0]}")
print(f"  b (learning exponent) = {df['b_true'].iloc[0]:.4f}")
print(f"  c (rate exponent) = {df['c_true'].iloc[0]:.4f}")
print(f"  Learning Rate = {2**df['b_true'].iloc[0]:.4f} ({2**df['b_true'].iloc[0]*100:.1f}%)")
print(f"  Rate Effect = {2**df['c_true'].iloc[0]:.4f} ({2**df['c_true'].iloc[0]*100:.1f}%)")

Training samples: 5
Test samples: 21

True parameters:
  T1 = 100
  b (learning exponent) = -0.1520
  c (rate exponent) = -0.1520
  Learning Rate = 0.9000 (90.0%)
  Rate Effect = 0.9000 (90.0%)


In [3]:
df_train

,scenario_id,n_lots,learning_rate,rate_effect,cv_error,replication,seed,program_id,total_program_lots,actual_correlation,...,lot_number,first_unit,last_unit,lot_quantity,lot_midpoint,log_midpoint,log_quantity,true_cost,observed_cost,log_observed_cost
0,2620.0,5.0,0.9,0.9,0.2,90.0,1987917204,10,26,0.980223,...,1,1,4,4,2.125088,0.753813,1.386294,72.230851,68.380513,4.225088
1,2620.0,5.0,0.9,0.9,0.2,90.0,1987917204,10,26,0.980223,...,2,5,11,7,7.691835,2.040159,1.945910,54.558596,60.394482,4.100898
2,2620.0,5.0,0.9,0.9,0.2,90.0,1987917204,10,26,0.980223,...,3,12,19,8,15.298691,2.727767,2.079442,48.156666,46.881073,3.847614
3,2620.0,5.0,0.9,0.9,0.2,90.0,1987917204,10,26,0.980223,...,4,20,32,13,25.683407,3.245845,2.564949,41.343344,32.595040,3.484160
4,2620.0,5.0,0.9,0.9,0.2,90.0,1987917204,10,26,0.980223,...,5,33,45,13,38.790661,3.658180,2.564949,38.831639,33.773370,3.519673


## The Cost Model

We're fitting a **learning curve model** commonly used in manufacturing cost estimation:

$$Y = T_1 \cdot X_1^b \cdot X_2^c$$

Where:
- $Y$ = per-unit cost
- $T_1$ = theoretical first unit cost
- $X_1$ = cumulative lot midpoint
- $X_2$ = lot quantity
- $b$ = learning exponent (negative = cost decreases with experience)
- $c$ = rate effect exponent (negative = cost decreases with larger lots)

### Economic Constraints

The **learning rate** $= 2^b$ and **rate effect** $= 2^c$ must be $\leq 1$ (i.e., $b, c \leq 0$).

A learning rate > 100% would mean costs *increase* with experience - economically nonsensical!

---
## Method 1: Ordinary Least Squares (OLS)

Log-transform to linearize: $\ln(Y) = \ln(T_1) + b \cdot \ln(X_1) + c \cdot \ln(X_2)$

In [4]:
# OLS with log-log transformation
ols = TransformedTargetRegressor(
    regressor=Pipeline([
        ('log', FunctionTransformer(np.log)),
        ('reg', LinearRegression()),
    ]),
    func=np.log,
    inverse_func=np.exp,
)

ols.fit(X_train, y_train)

# Extract coefficients
ols_b, ols_c = ols.regressor_.named_steps['reg'].coef_
ols_t1 = np.exp(ols.regressor_.named_steps['reg'].intercept_)
ols_lr = 2 ** ols_b  # Learning rate
ols_re = 2 ** ols_c  # Rate effect

print("OLS Results:")
print(f"  T1 = {ols_t1:.2f}")
print(f"  b = {ols_b:.4f}")
print(f"  c = {ols_c:.4f}")
print(f"  Learning Rate = {ols_lr:.4f} ({ols_lr*100:.1f}%)")
print(f"  Rate Effect = {ols_re:.4f} ({ols_re*100:.1f}%)")
print(f"\n  Train R² = {ols.score(X_train, y_train):.4f}")
print(f"  Test R² = {ols.score(X_test, y_test):.4f}")

# Check validity
ols_valid = (ols_lr <= 1) and (ols_re <= 1)
if not ols_valid:
    print(f"\n  ⚠️  INVALID: Learning Rate or Rate Effect > 100%!")

OLS Results:
  T1 = 195.00
  b = 0.0273
  c = -0.7140
  Learning Rate = 1.0191 (101.9%)
  Rate Effect = 0.6096 (61.0%)

  Train R² = 0.8851
  Test R² = -0.1843

  ⚠️  INVALID: Learning Rate or Rate Effect > 100%!


---
## Method 2: Ridge Regression with Cross-Validation (RidgeCV)

Adds L2 penalization to reduce overfitting, but still no economic constraints.

In [ ]:
# Ridge with CV for alpha selection
ridge = TransformedTargetRegressor(
    regressor=Pipeline([
        ('log', FunctionTransformer(np.log)),
        ('reg', RidgeCV(
            alphas=np.logspace(-5, 5, 11),
            fit_intercept=True,
            store_cv_results=True  # Fixed: was store_cv_values
        ))
    ]),
    func=np.log,
    inverse_func=np.exp,
)

ridge.fit(X_train, y_train)

# Extract coefficients
ridge_b, ridge_c = ridge.regressor_.named_steps['reg'].coef_
ridge_t1 = np.exp(ridge.regressor_.named_steps['reg'].intercept_)
ridge_lr = 2 ** ridge_b
ridge_re = 2 ** ridge_c
ridge_alpha = ridge.regressor_.named_steps['reg'].alpha_

print("RidgeCV Results:")
print(f"  Selected alpha = {ridge_alpha:.6f}")
print(f"  T1 = {ridge_t1:.2f}")
print(f"  b = {ridge_b:.4f}")
print(f"  c = {ridge_c:.4f}")
print(f"  Learning Rate = {ridge_lr:.4f} ({ridge_lr*100:.1f}%)")
print(f"  Rate Effect = {ridge_re:.4f} ({ridge_re*100:.1f}%)")
print(f"\n  Train R² = {ridge.score(X_train, y_train):.4f}")
print(f"  Test R² = {ridge.score(X_test, y_test):.4f}")

# Check validity
ridge_valid = (ridge_lr <= 1) and (ridge_re <= 1)
if not ridge_valid:
    print(f"\n  ⚠️  INVALID: Learning Rate or Rate Effect > 100%!")

---
## Method 3: PCReg - Constraints Only (No Penalization)

Uses the custom prediction function with bounded optimization to enforce:
- $b \in [-0.5, 0]$ → Learning Rate between 71% and 100%
- $c \in [-0.5, 0]$ → Rate Effect between 71% and 100%

In [6]:
def unit_space_prediction_fn(X: np.ndarray, params: np.ndarray) -> np.ndarray:
    """
    Learning curve prediction function for unit-space fitting.
    
    Y = T1 * X1^b * X2^c
    """
    T1, b, c = params[0], params[1], params[2]
    return T1 * (X[:, 0] ** b) * (X[:, 1] ** c)

In [8]:
# PCReg with constraints only (alpha=0 means no penalization)
pc_constrained = pcreg.PCRegression(
    coef_names=['T1', 'b', 'c'],
    bounds={'T1': (0, None), 'b': (-0.5, 0), 'c': (-0.5, 0)},
    prediction_fn=unit_space_prediction_fn,
    fit_intercept=False,
    x0=[1, 0, 0],
    loss='sspe',
    alpha=0,  # No penalization
    safe_mode=True
)

pc_constrained.fit(X_train, y_train)

# Extract coefficients
pc_con_t1 = pc_constrained.coef_[0]
pc_con_b = pc_constrained.coef_[1]
pc_con_c = pc_constrained.coef_[2]
pc_con_lr = 2 ** pc_con_b
pc_con_re = 2 ** pc_con_c

print("PCReg (Constraints Only) Results:")
print(f"  T1 = {pc_con_t1:.2f}")
print(f"  b = {pc_con_b:.4f}")
print(f"  c = {pc_con_c:.4f}")
print(f"  Learning Rate = {pc_con_lr:.4f} ({pc_con_lr*100:.1f}%)")
print(f"  Rate Effect = {pc_con_re:.4f} ({pc_con_re*100:.1f}%)")
print(f"\n  Train R² = {pc_constrained.score(X_train, y_train):.4f}")
print(f"  Test R² = {pc_constrained.score(X_test, y_test):.4f}")

# Check validity (should always be valid due to constraints)
pc_con_valid = (pc_con_lr <= 1) and (pc_con_re <= 1)
print(f"\n  ✓ Constraints satisfied: {pc_con_valid}")

PCReg (Constraints Only) Results:
  T1 = 152.33
  b = -0.0590
  c = -0.5000
  Learning Rate = 0.9599 (96.0%)
  Rate Effect = 0.7071 (70.7%)

  Train R² = 0.8873
  Test R² = -0.2138

  ✓ Constraints satisfied: True


---
## Method 4: PCReg-GCV (Penalization + Constraints)

Combines:
- **Economic constraints** (bounds on parameters)
- **Elastic Net penalization** (L1 + L2) with GCV-based hyperparameter selection
- **Penalty exclusion** for T1 (we don't want to shrink the intercept)

In [9]:
# PCReg with GCV for hyperparameter selection
pc_gcv = pcreg.PenalizedConstrainedCV(
    coef_names=['T1', 'b', 'c'],
    bounds={'T1': (0, None), 'b': (-0.5, 0), 'c': (-0.5, 0)},
    prediction_fn=unit_space_prediction_fn,
    fit_intercept=False,
    x0=[1, 0, 0],
    loss='sspe',
    alphas=np.logspace(-5, 0, 10), # Penalty
    l1_ratios=[0.0, 0.5, 1.0], # 
    cv=3,
    selection='gcv',
    penalty_exclude=['T1'],  # Don't penalize the intercept
    n_jobs=1,
    verbose=0,
    safe_mode=True
)

pc_gcv.fit(X_train, y_train)

# Extract coefficients
pc_gcv_t1 = pc_gcv.coef_[0]
pc_gcv_b = pc_gcv.coef_[1]
pc_gcv_c = pc_gcv.coef_[2]
pc_gcv_lr = 2 ** pc_gcv_b
pc_gcv_re = 2 ** pc_gcv_c

print("PCReg-GCV Results:")
print(f"  Selected alpha = {pc_gcv.alpha_:.6f}")
print(f"  Selected l1_ratio = {pc_gcv.l1_ratio_:.2f}")
print(f"  T1 = {pc_gcv_t1:.2f}")
print(f"  b = {pc_gcv_b:.4f}")
print(f"  c = {pc_gcv_c:.4f}")
print(f"  Learning Rate = {pc_gcv_lr:.4f} ({pc_gcv_lr*100:.1f}%)")
print(f"  Rate Effect = {pc_gcv_re:.4f} ({pc_gcv_re*100:.1f}%)")
print(f"\n  Train R² = {pc_gcv.score(X_train, y_train):.4f}")
print(f"  Test R² = {pc_gcv.score(X_test, y_test):.4f}")

# Check validity (should always be valid due to constraints)
pc_gcv_valid = (pc_gcv_lr <= 1) and (pc_gcv_re <= 1)
print(f"\n  ✓ Constraints satisfied: {pc_gcv_valid}")

PCReg-GCV Results:
  Selected alpha = 1.000000
  Selected l1_ratio = 0.00
  T1 = 99.23
  b = -0.1984
  c = -0.1345
  Learning Rate = 0.8715 (87.2%)
  Rate Effect = 0.9110 (91.1%)

  Train R² = 0.8767
  Test R² = -0.2373

  ✓ Constraints satisfied: True


---
## Results Comparison

In [10]:
# True values
true_t1 = df['T1_true'].iloc[0]
true_b = df['b_true'].iloc[0]
true_c = df['c_true'].iloc[0]
true_lr = 2 ** true_b
true_re = 2 ** true_c

# Create comparison dataframe
results = pd.DataFrame({
    'Method': ['True', 'OLS', 'RidgeCV', 'PCReg (Const.)', 'PCReg-GCV'],
    'T1': [true_t1, ols_t1, ridge_t1, pc_con_t1, pc_gcv_t1],
    'b': [true_b, ols_b, ridge_b, pc_con_b, pc_gcv_b],
    'c': [true_c, ols_c, ridge_c, pc_con_c, pc_gcv_c],
    'Learning Rate': [true_lr, ols_lr, ridge_lr, pc_con_lr, pc_gcv_lr],
    'Rate Effect': [true_re, ols_re, ridge_re, pc_con_re, pc_gcv_re],
    'Train R²': [np.nan, ols.score(X_train, y_train), ridge.score(X_train, y_train), 
                 pc_constrained.score(X_train, y_train), pc_gcv.score(X_train, y_train)],
    'Test R²': [np.nan, ols.score(X_test, y_test), ridge.score(X_test, y_test),
                pc_constrained.score(X_test, y_test), pc_gcv.score(X_test, y_test)],
    'Valid': ['Yes', 'Yes' if ols_valid else 'NO', 'Yes' if ridge_valid else 'NO', 
              'Yes' if pc_con_valid else 'NO', 'Yes' if pc_gcv_valid else 'NO']
})

results.set_index('Method', inplace=True)
results.round(4)

NameError: name 'ridge_t1' is not defined

In [ ]:
# Visualize the learning rates
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

methods = ['True', 'OLS', 'RidgeCV', 'PCReg\n(Const.)', 'PCReg-GCV']
learning_rates = [true_lr, ols_lr, ridge_lr, pc_con_lr, pc_gcv_lr]
rate_effects = [true_re, ols_re, ridge_re, pc_con_re, pc_gcv_re]
colors = ['green', 'red' if ols_lr > 1 else 'blue', 'red' if ridge_lr > 1 else 'blue', 'blue', 'blue']

# Learning Rate
ax1 = axes[0]
bars1 = ax1.bar(methods, learning_rates, color=colors, edgecolor='black', alpha=0.7)
ax1.axhline(y=1.0, color='red', linestyle='--', linewidth=2, label='Max Valid (100%)')
ax1.set_ylabel('Learning Rate', fontsize=12)
ax1.set_title('Learning Rate by Method', fontsize=14)
ax1.set_ylim(0, max(learning_rates) * 1.15)
ax1.legend()
for bar, lr in zip(bars1, learning_rates):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
             f'{lr*100:.1f}%', ha='center', va='bottom', fontsize=10)

# Rate Effect
colors2 = ['green', 'red' if ols_re > 1 else 'blue', 'red' if ridge_re > 1 else 'blue', 'blue', 'blue']
ax2 = axes[1]
bars2 = ax2.bar(methods, rate_effects, color=colors2, edgecolor='black', alpha=0.7)
ax2.axhline(y=1.0, color='red', linestyle='--', linewidth=2, label='Max Valid (100%)')
ax2.set_ylabel('Rate Effect', fontsize=12)
ax2.set_title('Rate Effect by Method', fontsize=14)
ax2.set_ylim(0, max(rate_effects) * 1.15)
ax2.legend()
for bar, re in zip(bars2, rate_effects):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
             f'{re*100:.1f}%', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.savefig('output_v2/motivating_example_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

---
## Model Diagnostics for PCReg-GCV

In [12]:
# Generate diagnostics report
diag = pcreg.ModelDiagnostics(pc_gcv, X_train, y_train)
report = diag.summary(bootstrap=True, n_bootstrap=100, include_alpha_trace=True)
report.plot_diagnostics()

In [ ]:
# Export HTML report
report.to_html("output_v2/motivating_example_diagnostics.html")
print("Diagnostics report saved to output_v2/motivating_example_diagnostics.html")

---
## Key Takeaways

1. **OLS can fail**: With limited/noisy data, OLS may produce economically impossible coefficients

2. **Penalization alone isn't enough**: RidgeCV helps with overfitting but doesn't guarantee valid coefficients

3. **Constraints ensure validity**: PCReg with bounds guarantees economically sensible results

4. **Combined approach is best**: PCReg-GCV provides:
   - Economic validity through constraints
   - Better generalization through penalization
   - Automatic hyperparameter tuning via GCV
   - Interpretable, defensible estimates

In [ ]:
# Final summary
print("="*70)
print("MOTIVATING EXAMPLE SUMMARY")
print("="*70)
print(f"\nTrue Learning Rate: {true_lr*100:.1f}%")
print(f"True Rate Effect: {true_re*100:.1f}%")
print()
print(f"OLS Learning Rate: {ols_lr*100:.1f}% {'⚠️ INVALID' if ols_lr > 1 else '✓'}")
print(f"OLS Rate Effect: {ols_re*100:.1f}% {'⚠️ INVALID' if ols_re > 1 else '✓'}")
print()
print(f"RidgeCV Learning Rate: {ridge_lr*100:.1f}% {'⚠️ INVALID' if ridge_lr > 1 else '✓'}")
print(f"RidgeCV Rate Effect: {ridge_re*100:.1f}% {'⚠️ INVALID' if ridge_re > 1 else '✓'}")
print()
print(f"PCReg (Constraints) Learning Rate: {pc_con_lr*100:.1f}% ✓")
print(f"PCReg (Constraints) Rate Effect: {pc_con_re*100:.1f}% ✓")
print()
print(f"PCReg-GCV Learning Rate: {pc_gcv_lr*100:.1f}% ✓")
print(f"PCReg-GCV Rate Effect: {pc_gcv_re*100:.1f}% ✓")
print("="*70)